<a href="https://colab.research.google.com/github/sgsayani/Sentiment-analysis_LSTM/blob/main/Sentimental_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np 
import pandas as pd 
import keras.losses
from keras.optimizers import SGD
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.np_utils import to_categorical
import re
from keras.backend import clear_session

In [ ]:
data = pd.read_csv('train.csv')
data =data.dropna()
print(data)

           textID                                               text  \
0      cb774db0d1                I`d have responded, if I were going   
1      549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2      088c60f138                          my boss is bullying me...   
3      9642c003ef                     what interview! leave me alone   
4      358bd9e861   Sons of ****, why couldn`t they put them on t...   
...           ...                                                ...   
27476  4eac33d1c0   wish we could come see u on Denver  husband l...   
27477  4f4c4fc327   I`ve wondered about rake to.  The client has ...   
27478  f67aae2310   Yay good for both of you. Enjoy the break - y...   
27479  ed167662a5                         But it was worth it  ****.   
27480  6f7127d9d7     All this flirting going on - The ATG smiles...   

                                           selected_text sentiment  
0                    I`d have responded, if I were going   neutral

In [ ]:
data.isnull().sum()

textID           0
text             0
selected_text    0
sentiment        0
dtype: int64

In [ ]:
data.head() 

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [ ]:
#list1=["textID","selected_text"]
#features = data[list1]
#print(features)

           textID                                      selected_text
0      cb774db0d1                I`d have responded, if I were going
1      549e992a42                                           Sooo SAD
2      088c60f138                                        bullying me
3      9642c003ef                                     leave me alone
4      358bd9e861                                      Sons of ****,
...           ...                                                ...
27476  4eac33d1c0                                             d lost
27477  4f4c4fc327                                      , don`t force
27478  f67aae2310                          Yay good for both of you.
27479  ed167662a5                         But it was worth it  ****.
27480  6f7127d9d7  All this flirting going on - The ATG smiles. Y...

[27480 rows x 2 columns]


In [ ]:
#class1=data["sentiment"]
#print(class1)

0         neutral
1        negative
2        negative
3        negative
4        negative
           ...   
27476    negative
27477    negative
27478    positive
27479    positive
27480     neutral
Name: sentiment, Length: 27480, dtype: object



**START FROM HERE**

In [6]:
data = pd.read_csv('train.csv')
# Keeping only the neccessary columns
data = data[['selected_text','sentiment']]
print(data)

                                           selected_text sentiment
0                    I`d have responded, if I were going   neutral
1                                               Sooo SAD  negative
2                                            bullying me  negative
3                                         leave me alone  negative
4                                          Sons of ****,  negative
...                                                  ...       ...
27476                                             d lost  negative
27477                                      , don`t force  negative
27478                          Yay good for both of you.  positive
27479                         But it was worth it  ****.  positive
27480  All this flirting going on - The ATG smiles. Y...   neutral

[27481 rows x 2 columns]


In [7]:
data = data[data.sentiment != "Neutral"]
data['selected_text'] = data['selected_text'].astype(str).apply(lambda x: x.lower())
data['selected_text'] = data['selected_text'].apply((lambda x:re.sub('[^a-zA-z0-9\s]','',x)))

 
print(data[ data['sentiment']=='positive'].size)
print(data[ data['sentiment']=='negative'].size)

for idx,row in data.iterrows():
  row[0]=row[0].replace('rt',' ')
max_features = 20000
tokenizer = Tokenizer(num_words = max_features,split=' ')
tokenizer.fit_on_texts(data['selected_text'].values)
x=tokenizer.texts_to_sequences(data['selected_text'].values)
x=pad_sequences(x)

17164
15562


In [8]:
y=pd.get_dummies(data['sentiment']).values

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=42)
clear_session()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


(24732, 34) (24732, 3)
(2749, 34) (2749, 3)


In [9]:
embed_dim = 128
lstm_out = 196

model = Sequential()

model.add(Embedding(max_features,embed_dim,input_length=x.shape[1]))

model.add(SpatialDropout1D(0.4))

model.add(LSTM(lstm_out,dropout=0.2,recurrent_dropout=0.2))

model.add(Dense(3,activation='softmax'))
#model.compile(loss='categorical_crossentropy', optimizer=SGD, metrics=['accuracy'])

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 34, 128)           2560000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 34, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 2,815,391
Trainable params: 2,815,391
Non-trainable params: 0
_________________________________________________________________


In [10]:
batch_size = 32
model.fit(x_train, y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
773/773 - 139s - loss: 0.5731 - accuracy: 0.7674 - 139s/epoch - 180ms/step
Epoch 2/10
773/773 - 116s - loss: 0.3596 - accuracy: 0.8713 - 116s/epoch - 150ms/step
Epoch 3/10
773/773 - 115s - loss: 0.2672 - accuracy: 0.9067 - 115s/epoch - 148ms/step
Epoch 4/10
773/773 - 112s - loss: 0.2165 - accuracy: 0.9260 - 112s/epoch - 145ms/step
Epoch 5/10
773/773 - 112s - loss: 0.1792 - accuracy: 0.9394 - 112s/epoch - 145ms/step
Epoch 6/10
773/773 - 113s - loss: 0.1521 - accuracy: 0.9475 - 113s/epoch - 147ms/step
Epoch 7/10
773/773 - 112s - loss: 0.1286 - accuracy: 0.9550 - 112s/epoch - 144ms/step
Epoch 8/10
773/773 - 112s - loss: 0.1120 - accuracy: 0.9622 - 112s/epoch - 145ms/step
Epoch 9/10
773/773 - 112s - loss: 0.1006 - accuracy: 0.9660 - 112s/epoch - 145ms/step
Epoch 10/10
773/773 - 114s - loss: 0.0879 - accuracy: 0.9699 - 114s/epoch - 148ms/step


In [ ]:
model.fit(x_test, y_test, epochs = 10, batch_size=batch_size, verbose = 2)

In [11]:
validation_size = 500

x_validate = x_test[-validation_size:]
y_validate = y_test[-validation_size:]
x_test = x_test[:-validation_size]
y_test = y_test[:-validation_size]
score,acc = model.evaluate(x_test, y_test, verbose = 2, batch_size = batch_size)

71/71 - 1s - loss: 0.6694 - accuracy: 0.8315 - 1s/epoch - 16ms/step
